### Obtaining financial data for Amazon

In [1]:
import requests
import config
import pandas as pd
from StringIO import StringIO

#### Company list

In [2]:
base = 'https://api.usfundamentals.com/v1/'
param = 'companies/xbrl?&format=json&token={}'.format(config.us_fun_key)

In [3]:
r = requests.get('{}{}'.format(base, param)).json()

In [4]:
companies = pd.DataFrame(r).set_index('name_latest')['company_id'].to_dict()

#### List of Amazon Indicators

In [73]:
indicators = pd.read_csv(StringIO(r.text))['indicator_id'].unique()

In [74]:
indicators

array(['EarningsPerShareDiluted'], dtype=object)

#### Earnings per share

In [54]:
base = 'https://api.usfundamentals.com/v1/indicators/xbrl?'
ind = 'indicators=EarningsPerShareDiluted&'
param = '{}companies=1018724&frequency=q&token={}'.format(ind, config.us_fun_key)

In [55]:
r = requests.get('{}{}'.format(base, param))

In [67]:
EPS = pd.read_csv(StringIO(r.text)).T[2:]

In [68]:
EPS['date'] = EPS.index.map(lambda x: pd.Period(x).end_time.date())
EPS = EPS.set_index('date').rolling(4).sum()

### Stock price history

In [10]:
base = 'https://www.quandl.com/api/v3/datatables/WIKI/PRICES.json?'
param = 'ticker=AMZN&qopts.columns=date,close&api_key={}'.format(config.quandl_key)

In [11]:
r = requests.get('{}{}'.format(base, param)).json()

In [46]:
df = pd.DataFrame(r['datatable']['data']).set_index(0)

In [49]:
idx = pd.date_range(df.index[0], df.index[-1])
df.index = pd.DatetimeIndex(df.index)
df = df.reindex(idx, fill_value=None).fillna(method='ffill')

In [70]:
PE = EPS.merge(df, left_index=True, right_index=True, how='left')

In [71]:
PE['PE_ratio'] = PE[1] / PE[0]

In [72]:
PE

,0,1,PE_ratio
date,,,
2011-03-31,NaN,180.130,NaN
2011-06-30,NaN,204.490,NaN
2011-09-30,NaN,216.230,NaN
2011-12-31,1.37,173.100,126.350365
2012-03-31,1.21,202.510,167.363636
2012-06-30,0.81,228.350,281.913580
2012-09-30,0.07,254.320,3633.142857
2012-12-31,-0.09,250.870,-2787.444444
2013-03-31,-0.19,266.490,-1402.578947
